<a href="https://colab.research.google.com/github/kessingtonosazee/GCP_Project_1/blob/master/Code_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
# Install packages
!pip install pulp
import math


In [73]:
# Import Libraries
import numpy as np
from pulp import LpMaximize, LpMinimize, LpProblem, LpVariable, lpSum


In [54]:
# Load data
wards = ["ward1", "ward2", "ward3", "ward4", "ward5", "ward6", "ward7", "ward8", "ward9", "ward10"]
max_beds_per_ward = {"ward1": 15, "ward2": 10, "ward3": 10, "ward4": 15, "ward5": 10, "ward6": 10, "ward7": 10, "ward8": 15, "ward9":10 , "ward10": 15}

In [69]:


# Assigning priority
def priority_levels(ward):
    # Define custom weights based on specific criteria
    weights = {"ward1":1.0,"ward2":0.5,"ward3":0.2,"ward4":0.5,"ward5":0.2,"ward6":0.2,"ward7":0.2,"ward8":0.5,"ward9":0.2,"ward10":0.5}
    return weights[ward]


In [70]:
# Simulate uncertain demand using a normal distribution
np.random.seed(42)  # Set seed for reproducibility
demand_mean = {"ward1": 10, "ward2": 5, "ward3": 5, "ward4": 10, "ward5": 5, "ward6": 5, "ward7": 4, "ward8": 10, "ward9": 5, "ward10": 10}
demand_std_dev = {"ward1": 3, "ward2": 2, "ward3": 1, "ward4": 3, "ward5":2 , "ward6": 1, "ward7": 2, "ward8": 2, "ward9": 2, "ward10": 3}
demand = {ward: max(0, int(math.ceil(np.random.normal(demand_mean[ward], demand_std_dev[ward])* priority_levels(ward)))) for ward in wards}

In [71]:
demand

{'ward1': 12,
 'ward2': 3,
 'ward3': 2,
 'ward4': 8,
 'ward5': 1,
 'ward6': 1,
 'ward7': 2,
 'ward8': 6,
 'ward9': 1,
 'ward10': 6}

In [74]:
# Priority for each ward (you can adjust these values)
# ward_priority = {"ward1": 2, "ward2": 1, "ward3": 3}

In [75]:
# Real-time bed availability
real_time_availability = {"ward1":12,"ward2":8,"ward3":7 ,"ward4":15 ,"ward5":7 ,"ward6":5,"ward7":10, "ward8":12, "ward9":6, "ward10":12}


In [97]:
# Create a linear programming problem
model_min = LpProblem(name="Healthcare_Bed_Minimum_Allocation", sense=LpMinimize)

In [98]:
# Decision Variables: Number of beds assigned to each ward
beds = {ward: LpVariable(name=f"{ward}_beds", lowBound=0, cat="Integer") for ward in wards}

In [100]:
# Objective Function: Minimize the total weighted number of beds
model_min += lpSum(priority_levels(ward) * beds[ward] for ward in wards), "Total_Weighted_Beds"

In [101]:
# Constraints: Demand must be met for each ward, and the number of beds cannot exceed the maximum for each ward
for ward in wards:
    model_min += beds[ward] >= demand[ward], f"Ward_Demand_{ward}"
    model_min += beds[ward] <= max_beds_per_ward[ward], f"Max_Bed_{ward}"
    model_min += beds[ward] <= real_time_availability[ward], f"Real_Time_Availability_{ward}"


In [102]:
# Solve the linear programming problem
model_min.solve()

1

In [103]:
# Display the results
if model_min.status == 1:  # Optimal solution found
    for ward in wards:
        print(f"Minimum number of beds for {ward}: {beds[ward].varValue}")
else:
    print("No optimal solution found.")


Minimum number of beds for ward1: 12.0
Minimum number of beds for ward2: 3.0
Minimum number of beds for ward3: 2.0
Minimum number of beds for ward4: 8.0
Minimum number of beds for ward5: 1.0
Minimum number of beds for ward6: 1.0
Minimum number of beds for ward7: 2.0
Minimum number of beds for ward8: 6.0
Minimum number of beds for ward9: 1.0
Minimum number of beds for ward10: 6.0


In [91]:
# Create a linear programming problem for maximization
model_max = LpProblem(name="Healthcare_Bed_Maximum_Allocation", sense=LpMaximize)

In [92]:
# Objective Function: Maximize the total weighted number of beds
model_max += lpSum(priority_levels(ward) * beds[ward] for ward in wards), "Total_Weighted_Beds"

In [93]:
# Constraints: Demand must be met for each ward, and the number of beds cannot exceed the maximum for each ward
for ward in wards:
    model_max += beds[ward] >= demand[ward], f"Ward_Demand_{ward}"
    model_max += beds[ward] <= max_beds_per_ward[ward], f"Max_Bed_{ward}"
    model_max += beds[ward] <= real_time_availability[ward], f"Real_Time_Availability_{ward}"

In [94]:
# Solve the linear programming problem
model_max.solve()

1

In [96]:
# Display the results
if model_max.status == 1:  # Optimal solution found
    for ward in wards:
        print(f"Maximum number of beds for {ward}: {beds[ward].varValue}")
else:
    print("No optimal solution found.")


Maximum number of beds for ward1: 12.0
Maximum number of beds for ward2: 8.0
Maximum number of beds for ward3: 7.0
Maximum number of beds for ward4: 15.0
Maximum number of beds for ward5: 7.0
Maximum number of beds for ward6: 5.0
Maximum number of beds for ward7: 10.0
Maximum number of beds for ward8: 12.0
Maximum number of beds for ward9: 6.0
Maximum number of beds for ward10: 12.0
